In [1]:
import pyodbc
import pandas as pd
import warnings
import urllib
 
 
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
 
warnings.filterwarnings('ignore', category=UserWarning)


In [9]:
import os
print (os.getenv('senha_bd_simples','senha_db'))

senha@clarify


In [13]:
senha = os.getenv('senha_bd_simples')

conn = pyodbc.connect(
    'DRIVER={SQL Server};' \
    'SERVER=den1.mssql7.gear.host;' \
    'DATABASE=vendasbicyclon;' \
    'UID=vendasbicyclon;' \
    f'PWD={senha};' \
    'MARS_Connection=Yes'
)  

In [4]:
# params = urllib.parse.quote_plus(
#     'DRIVER={SQL Server};' \
#     'SERVER=den1.mssql7.gear.host;' \
#     'DATABASE=vendasbicyclon;' \
#     'UID=vendasbicyclon;' \
#     'PWD=senha@clarify;' \
#     'MARS_Connection=Yes'
# )
 
# engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')
 

In [14]:
def exe_query(query):
    return pd.read_sql(query, conn)
 
lista_resposta = []
def exe_varias_querys(*args):
    for consulta in args:
        lista_resposta.append(pd.read_sql(consulta,conn))
    return lista_resposta
 

In [ ]:
spark: SparkSession = SparkSession.builder \
.master("local[2]") \
.appName("Teste") \
.getOrCreate()
 

In [15]:
q1 = "select * from clientes"
q2 = "select * from produtos"
 
exe_varias_querys(q1, q2)

[    idCliente              nomeCliente   cidadeCliente
 0           1           Leonardo Alves  Rio de Janeiro
 1           2      Ana Clara Rodrigues  Rio de Janeiro
 2           3      Felipe Gomes Santos       São Paulo
 3           4     Camila Oliveira Lima       São Paulo
 4           5    Thiago Costa Ferreira  Belo Horizonte
 5           6    Beatriz Martins Rocha  Belo Horizonte
 6           7       Gabriel Souza Lima        Curitiba
 7           8     Larissa Andrade Melo        Curitiba
 8           9   Ricardo Henrique Silva        Salvador
 9          10    Juliana Pereira Costa        Salvador
 10         11     Eduardo Alves Mendes        Brasília
 11         12   Carolina Nunes Ribeiro        Brasília
 12         13   Matheus Gonçalves Lima          Recife
 13         14     Amanda Barbosa Souza          Recife
 14         15      Bruno Dias Carvalho       Fortaleza
 15         16    Isabela Freitas Rocha       Fortaleza
 16         17  Leonardo Castro Almeida    Porto

In [16]:
q = """ 
select 
p.nomeProduto,
count(i.idItem) as total_pedidos,
sum(i.quantidade) as qtd_vendido
from produtos p 
    join itensPedido i on p.idProduto = i.fkProduto
    group by p.nomeProduto 
"""
exe_query(q)

,nomeProduto,total_pedidos,qtd_vendido
0,Caloi Elite Carbon,26,45
1,Caloi Urbam,35,68
2,Capacete Giro Register,35,62
3,Cassete Shimano 11v,39,89
4,Freio a Disco Hidráulico Shimano,36,72
5,Grupo Shimano Deore,37,80
6,Luva Ciclismo Gel,35,62
7,Luz Traseira Recarregável,28,54
8,Oggi Lite Tour,33,72
9,Sense Impact Pro,25,53


CONSOLIDADO - VIEW

In [21]:
q_vw = """
if OBJECT_ID('vwFatoVendas','V') is not null
    drop view vwFatoVendas

exec('
    create view vwFatoVendas as 
    select
        p.idPedido,
        p.dataPedido,
        c.nomeCliente,
        v.nomeVendedor,
        r.nomeRegional,
        pr.nomeProduto,
        l.nomeLinhaProduto,
        i.quantidade,
        i.desconto,
        (i.quantidade*(i.precoUnitario - i.desconto)) as valor_total

    from pedidos 
        join clientes c on p.fkCliente = c.idPedido
        join vendedores v on p.fkVendedor = v.IdVendedor
        join regionais r on v.fkRegional = r.idRegional
        join itensPedido i on i.fkPedido = p.idPedido
        join produtos pr on i.fkProduto = p.idProduto
        join linhasProdutos l on p.fkLinhaProduto = l.idLinhaProduto 
')
"""




In [22]:
cursor = conn.cursor()
cursor.execute(q_vw)
conn.commit()

df_fatovendas = pd.read_sql('select * from vwFatoVendas',conn)

df_fatovendas.columns

ProgrammingError: ('42S22', '[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name \'idPedido\'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.fkCliente" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.fkVendedor" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.idPedido" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.idProduto" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.fkLinhaProduto" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.idPedido" could not be bound. (4104); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]The multi-part identifier "p.dataPedido" could not be bound. 